In [96]:
from random import randint
from collections import Counter

In [2]:
def HammingDistance(str1, str2):
    i = 0
    count = 0
 
    while(i < len(str1)):
        if(str1[i] != str2[i]):
            count += 1
        i += 1
    return count

In [181]:
def score(motifs):
    score = 0
    for i in range(len(motifs[0])):
        motif = ''.join([motifs[j][i] for j in range(len(motifs))])
        score += min([HammingDistance(motif, seq*len(motif)) for seq in 'ACGT'])
    return score

def profile_most_probable_kmer(dna, k, prof):
    nuc_loc = {nucleotide:index for index,nucleotide in enumerate('ACGT')}
    max_prob = [-1, None]
    for i in range(len(dna)-k+1):
        current_prob = 1
        for j, nucleotide in enumerate(dna[i:i+k]):
            current_prob *= prof[j][nuc_loc[nucleotide]]
        if current_prob > max_prob[0]:
            max_prob = [current_prob, dna[i:i+k]]
    return max_prob[1]

def profile_with_pseudocounts(motifs,k):
    prof = []
    for i in range(len(motifs[0])):
        freq = Counter([motifs[j][i] for j in range(len(motifs))])
        for j in ['A','C','G','T']:
            if j not in freq:
                freq[j] = 1
            else:
                freq[j] += 1
        t = []
        g = sum(freq.values())
        for i in ['A','C','G','T']:
            t.append(freq[i] / g)
        prof.append(t)
    return prof


def randomized_motif_search(dna,k,t):
    rand_ints = [randint(0,len(dna[0])-k) for a in range(t)]
    motifs = [dna[i][r:r+k] for i,r in enumerate(rand_ints)]
    best_score = [score(motifs), motifs]
    while True:
        prof = profile_with_pseudocounts(motifs,k)
        motifs = [profile_most_probable_kmer(seq,k,prof) for seq in dna]
        current_score = score(motifs)
        if current_score < best_score[0]:
            best_score = [current_score, motifs]
        else:
            return best_score
        
def gibbs_sampler(dna,k,t,N):
    rand_ints = [randint(0,len(dna[0])-k) for a in range(t)]
    motifs = [dna_list[i][r:r+k] for i,r in enumerate(rand_ints)]
    best_score = [score(motifs), motifs]
    for i in range(N):
        r = randint(0,t-1)
        current_profile = profile_with_pseudocounts([motif for index, motif in enumerate(motifs) if index!=r],k)
        motifs = [profile_most_probable_kmer(dna[index],k,current_profile) if index == r else motif for index,motif in enumerate(motifs)]
        current_score = score(motifs)
        if current_score < best_score[0]:
            best_score = [current_score, motifs]

    return best_score

def rms(k,t,N,dna_list):
    best_motifs = [k*t, None]
    for repeat in range(N):
        current_motifs = randomized_motif_search(dna_list,k,t)
        if current_motifs[0] < best_motifs[0]:
            best_motifs = current_motifs

    print(best_motifs[1])

def gibbs(k,t,N,dna_list):
    best_motifs = [k*t, None]
    for repeat in range(30):
        current_motifs = gibbs_sampler(dna_list,k,t,N)
        if current_motifs[0] < best_motifs[0]:
            best_motifs = current_motifs

    print(best_motifs[1])
    
k, t, N = 15, 10, 1000

In [182]:
with open('motif1.txt') as input_data:
    k,t = map(int, input_data.readline().split())
    dna_list = [line.strip() for line in input_data.readlines()]

rms(k,t,N,dna_list)

['TTTTCTC', 'GGTTCTA', 'AGTTATC', 'AGTTCTA', 'AGATCTA']


In [183]:
with open('motif2.txt') as input_data:
    k,t,N = map(int, input_data.readline().split())
    dna_list = [line.strip() for line in input_data.readlines()]

gibbs(k,t,N,dna_list)

['ATTTTTC', 'GGTTCTA', 'AGTTATC', 'AGTTCTA', 'AGATCTA']


In [28]:
with open('motif_dataset.txt') as input_data:
    dna_list = [line.strip() for line in input_data.readlines()]

rms(k,t,1000,dna_list)
rms(k,t,10000,dna_list)
rms(k,t,100000,dna_list)
gibbs(k,t,1000,dna_list)
gibbs(k,t,2000,dna_list)
gibbs(k,t,10000,dna_list)

['CGAGAGG', 'CGAGAGA', 'GGGGAGA', 'CGGGAGA', 'CGGGAGA', 'CGGGAGA', 'CGAGAGC', 'CGAGAGA', 'CGAGAGA', 'CGAGAGC']
['AAAGAAA', 'AAATAAA', 'AAATAAA', 'AAAAAAA', 'AAATAAA', 'AAAGAAT', 'AAAGAAA', 'AAAGGAA', 'AAAGAGA', 'AGAGAAA']
['CGAGAGG', 'CGAGAGA', 'GGGGAGA', 'CGGGAGA', 'CGGGAGA', 'CGGGAGA', 'CGAGAGC', 'CGAGAGA', 'CGAGAGA', 'CGAGAGC']
['AGTTCCG', 'AGCTCCG', 'AGCTTCG', 'AGCTCAG', 'AGCTCCG']
['AAAGAAA', 'AAATAAA', 'AAATAAA', 'AAAAAAA', 'AAATAAA']
['GGACTCG', 'GGACTCG', 'GGACTCG', 'CGACTCG', 'GGACTCC']


In [59]:
with open('RandomizedMotifSearch/inputs/input_1.txt') as input_data:
    k,t = map(int, input_data.readline().split())
    dna_list = [line.strip() for line in input_data.readlines()]
rms(k,t,N,dna_list)
with open('RandomizedMotifSearch/outputs/output_1.txt') as output_data:
    dna_list_1 = [line.strip() for line in output_data.readlines()]
print(dna_list_1)

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']


In [58]:
for i in range(1,5):
    with open('RandomizedMotifSearch/inputs/input_'+str(i)+'.txt') as input_data:
        k,t = map(int, input_data.readline().split())
        dna_list = [line.strip() for line in input_data.readlines()]
    rms(k,t,N,dna_list)

['TCTCGGGG', 'TGTAAGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']
['CGATAA', 'GGTTAA', 'GGTATA', 'GGTTAA', 'GGTTAC', 'GGTTAA', 'GGCCAA', 'GGTTAA']
['TTAACC', 'ATAACT', 'TTAACC', 'TGAAGT', 'TTAACC', 'TTAAGC', 'TTAACC', 'TGAACA']
['CATGGGGAAAACTGA', 'CCTCTCGATCACCGA', 'CCTATAGATCACCGA', 'CCGATTGATCACCGA', 'CCTTGTGCAGACCGA', 'CCTTGCCTTCACCGA', 'CCTTGTTGCCACCGA', 'ACTTGTGATCACCTT', 'CCTTGTGATCAATTA', 'CCTTGTGATCTGTGA', 'CCTTGTGATCACTCC', 'AACTGTGATCACCGA', 'CCTTAGTATCACCGA', 'CCTTGTGAAATCCGA', 'CCTTGTCGCCACCGA', 'TGTTGTGATCACCGC', 'CACCGTGATCACCGA', 'CCTTGGTTTCACCGA', 'CCTTTGCATCACCGA', 'CCTTGTGATTTACGA']


In [53]:
for i in range(1,5):
    with open('rand_testcases/test_'+str(i)+'.txt') as input_data:
        k,t = map(int, input_data.readline().split())
        dna_list = [line.strip() for line in input_data.readlines()]
    rms(k,t,N,dna_list)
    
    
    
    

['TGATAAGCATGAATA', 'TAAACCTCCTCACTT', 'TAACAATCCTCGAGT', 'TAGGGATCCTCACTT', 'TAACAATCCAGCCTT', 'TAACACAACTCACTT', 'TAACTGCCCTCACTT', 'TAACACTCCTCCCTG', 'ACTCAATCCTCACTT', 'TAACAATGTGCACTT', 'TAACAAAGTTCACTT', 'TAACAATCCTGCGTT', 'TAACAATCTATACTT', 'TGGTAATCCTCACTT', 'TAACAAAGATCACTT', 'TAACCCGCCTCACTT', 'CAACAATCCTCACGA', 'TAACACAGCTCACTT', 'TAAATCTCCTCACTT', 'TAACAATCCTCAGGC']
['TGATGCTAGAGCACC', 'TGCACTTATGGCACT', 'TGCGGGCCTGGCACT', 'TGCACGTATGGCACT', 'TGCGGATAAATCACT', 'TGCGGATATGGCGAA', 'TGATCATATGGCACT', 'TGCGTCCATGGCACT', 'TGCGGCGCTGGCACT', 'TGCGGAGCCGGCACT', 'TGCGGATTCAGCACT', 'GCCGGATATGGCACG', 'TGCGGATATGCGGCT', 'GTGGGATATGGCACT', 'TTGAGATATGGCACT', 'CGCGGATATGGCAGA', 'TGCGGATATCTAACT', 'TGCGATGATGGCACT', 'TGCGGACCGGGCACT', 'TGCGGATATGGTCAT']
['ATACCACGTATTAAG', 'ATATGACGAAACAAA', 'ATAAGGAGAAACAAA', 'CAAACCCGAAACAAG', 'ATAACCCGACCGAAA', 'ATAACCTCGAACAAA', 'ATCTTCCGAAACAAA', 'ATATGGCGAAACAAA', 'ATAACGTAAAACAAA', 'ATAAATGGAAACAAA', 'ATAACCCGAAACTGG', 'ATAACCTAGAACAAA', 'ATAACCCG

In [180]:
for i in range(1,5):
    with open('gibbs_testcases/test_'+str(i)+'.txt') as input_data:
        k,t,N = map(int, input_data.readline().split())
        dna_list = [line.strip() for line in input_data.readlines()]
    gibbs(k,t,N,dna_list)
    

['GGTACGTGTAGACCT', 'GTAACGAGCATCCGA', 'GACCAAACTTATTCT', 'ATGAATGAACCCGGA', 'TGTACGCAGCATAAC', 'CGTCGTCCGCGTCGC', 'CAATATAGACTAAAT', 'GAAGTCATATTCTGG', 'CATTCTGTCTACTCA', 'AAGATAGCCGAACCC', 'AGTGGGACATTTGAG', 'ATTTATCGGGGTTAG', 'AATGCCATTACCTGG', 'TGATCCACAGTGTAG', 'GGTAGTACAAAGCGG', 'ATTGATATATGTTAT', 'TAAAAAAGATAGCCA', 'GTGAGTAAGTAATGG', 'ATTAATCTACATTCG', 'CCTGACAGTCTCGGC']
['TCAGTCTGGCCTTGC', 'CGATCGACTGGAAAG', 'CATGCCCCCACCACA', 'CGAAGAGGCGCGAAA', 'GGTTGCCCCACTTGA', 'CGGTAAGGGGTACGG', 'CGCGACCAAAAAATA', 'CTGCCAGTCAGCTTA', 'TGCTAAGTATTGCGC', 'GCCCCCACCGGCACA', 'CTAGCCCCCCGTTAC', 'ACAGCTCCCACACGA', 'CAAACGGTGAGCCCC', 'TGCCCGGCCAGACTC', 'CGAGGTTCCTATCCG', 'TTCGGCGCCAGAATC', 'TAAATGCATGCACCA', 'GTCCGCTTAAAACAC', 'CAGGCATCCAGCGTG', 'TTCCGAAGCCTACTA']
['ACCCGCTGAGTCGGC', 'CTCGCGAGGTTTGGC', 'TGCGCATAGGTGAAA', 'TTAAGCTTGTTTCGG', 'TACTATCTAATTTGT', 'ATGTCGAAGTTAATG', 'CGCTCATGCGGTTAC', 'CAGGCGCGCCGCACC', 'CGCGTGACATTCTAC', 'ACCGGACAACCGAGT', 'CTGGCGATCGCCCGG', 'CGCTCGAGCGGACGC', 'CTTCAGGC

In [ ]:
def profile(motifs):
    prof = []
    for i in range(len(motifs[0])):
        col = ''.join([motifs[j][i] for j in range(len(motifs))])
        prof.append([float(col.count(i))/float(len(col)) for i in 'ACGT'])
    return prof

 